In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to your Pokémon dataset folder
dataset_path = "pokemon2"

# Specify the image size and batch size
img_size = (224, 224)
batch_size = 32

# Create an ImageDataGenerator for data augmentation and normalization
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% of the data will be used for validation
)

# Load and preprocess the dataset using the generator
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # Assuming you have multiple classes
    subset='training'  # Specify that this is the training set
)

val_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Specify that this is the validation set
)

# Create the CNN model
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))  # Optional dropout for regularization
model.add(layers.Dense(len(train_generator.class_indices), activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Display the model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size
)


Found 1118 images belonging to 8 classes.
Found 275 images belonging to 8 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                       

2024-04-15 14:45:38.367678: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


30/34 [=========================>....] - ETA: 2s - loss: 2.0914 - accuracy: 0.3173

/opt/anaconda3/lib/python3.11/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


34/34 [==============================] - 29s 843ms/step - loss: 2.0160 - accuracy: 0.3297 - val_loss: 1.2656 - val_accuracy: 0.5742
Epoch 2/10
18/34 [==============>...............] - ETA: 10s - loss: 1.3286 - accuracy: 0.5469

KeyboardInterrupt: 

In [36]:
model.save('modelfinal2.h5')

In [4]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Path to your trained model file (.h5)
model_path = "modelfinal2.h5"

# Path to the input image you want to classify
input_image_path = "test5.png"

# Load the trained model
model = tf.keras.models.load_model(model_path)

# Load and preprocess the input image
img = image.load_img(input_image_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalize the pixel values to be between 0 and 1

# Make predictions
predictions = model.predict(img_array)

# Get the predicted class index
predicted_class_index = np.argmax(predictions[0])

# Get the class labels from the generator
class_labels = list(train_generator.class_indices.keys())

# Get the predicted class label
predicted_class_label = class_labels[predicted_class_index]

# Display the prediction
print("Predicted class:", predicted_class_label)



1/1 [==============================] - 0s 106ms/step
Predicted class: Charmeleon


In [5]:
# Evaluate the model on the validation dataset
val_loss, val_accuracy = model.evaluate(val_generator)

# Print the validation accuracy
print("Validation Accuracy:", val_accuracy)


9/9 [==============================] - 5s 515ms/step - loss: 0.3745 - accuracy: 0.8800
Validation Accuracy: 0.8799999952316284
